In [7]:
import pandas as pd
import urllib.parse

In [2]:
queries = pd.read_csv('/Users/vli/Downloads/query_sample - query_sample.csv')

In [4]:
queries=queries[['query','len', 'len_bucket']]

In [5]:
queries

,query,len,len_bucket
0,runners,1,1
1,roast caps,2,1
2,mayver smooth,2,1
3,beef rub,2,1
4,natural lollies,2,1
...,...,...,...
2824,sea salt and vin,4,3
2825,cup of soup dutch curry,5,3
2826,beef and onion sau,4,3
2827,natures gift dry food,4,3


In [36]:
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By

driver_path = 'chromedriver_mac_arm64/chromedriver'

driver = Chrome(executable_path=driver_path)

/var/folders/w2/cc31wnk17sb792wmhqk254h80000gn/T/ipykernel_61934/653844092.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = Chrome(executable_path=driver_path)


In [37]:
def get_search_count(query):
    driver.get(f'https://www.coles.com.au/search?q='+urllib.parse.quote(query))
    xpath = '//div[@class="sc-4a711676-1 bJevuC coles-targeting-SearchPagesTopBarNumberOfResults"]'
    link_elements = driver.find_elements(By.XPATH,xpath)
    if len(link_elements)>0:
        result_cnt = link_elements[0].text
        result_cnt = result_cnt.split('of')[1].strip().split()[0]
    else:
        result_cnt = 0
    return result_cnt
    

In [38]:
queries['coles_cnt'] = queries['query'].apply(get_search_count)

In [40]:
queries.to_csv('query_sample_with_coles_search_cnts.csv')

In [41]:
wws_queries = pd.read_csv('../data/wws_queries.csv')

In [43]:
wws_queries.columns = ['wws_search_cnt','title','query']

In [45]:
queries = pd.merge(queries,wws_queries,how='left', on='query')

In [47]:
queries=queries[['query','len','len_bucket','coles_cnt','wws_search_cnt']]

In [56]:
def get_cnt(text):
    text = str(text)
    if text is None or text == 'nan':
        return 0
    elif 'of' in text:
        return text.split('of')[1].strip().split()[0]
    else:
        return text.strip().split()[0]

In [57]:
queries['wws_search_cnt'] = queries['wws_search_cnt'].apply(get_cnt)

/var/folders/w2/cc31wnk17sb792wmhqk254h80000gn/T/ipykernel_61934/347865827.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  queries['wws_search_cnt'] = queries['wws_search_cnt'].apply(get_cnt)


In [64]:
queries = queries[~((queries['coles_cnt']==0) & (queries['wws_search_cnt']==0))]

In [68]:
queries['coles_url'] = queries['query'].apply(lambda x: f'https://www.coles.com.au/search?q='+urllib.parse.quote(x))

/var/folders/w2/cc31wnk17sb792wmhqk254h80000gn/T/ipykernel_61934/2671867515.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  queries['coles_url'] = queries['query'].apply(lambda x: f'https://www.coles.com.au/search?q='+urllib.parse.quote(x))


In [69]:
queries.to_csv('query_sample.csv',index=False)

In [70]:
queries = pd.read_csv('query_sample.csv')

In [72]:
queries = queries.groupby('len_bucket', group_keys=False).apply(lambda x: x.sample(300))

In [73]:
queries.to_csv('final_query_sample.csv',index=False)